In [12]:
### Import de bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import LabelEncoder, OneHotEncoder



In [13]:
# Caminho do arquivo no projeto
pasta_arquivo = '/mnt/c/Users/Isabel/Documents/bootcamp_CDD/avanti-bootcamp-cdd/data/raw/diamonds.csv'

#Leitura do conjunto de dados 
df = pd.read_csv(pasta_arquivo, delimiter=',')
df.head(10)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
5,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,338,4.00,4.05,2.39


In [ ]:
## 1. Tratamento de dados discrepantes (outliers)

In [17]:
# Função para identificar e remover outliers usando IQR e imprimir a quantidade de outliers
def identify_and_count_outliers(df, cols):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Selecionando os outliers da coluna atual
        col_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        
        # Imprimindo apenas a quantidade de outliers por coluna
        print(f"Coluna: {col}")
        print(f"Quantidade de outliers: {col_outliers.shape[0]}\n")
    
    # Removendo os outliers do DataFrame principal
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

# Colunas que queremos verificar quanto aos outliers
cols_to_check = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
df_cleaned = identify_and_count_outliers(df, cols_to_check)

# Verificando o resultado após a remoção dos outliers
print("Resumo estatístico do DataFrame após remoção dos outliers:")
print(df_cleaned.describe())

Coluna: carat
Quantidade de outliers: 1889

Coluna: depth
Quantidade de outliers: 2545

Coluna: table
Quantidade de outliers: 605

Coluna: price
Quantidade de outliers: 3540

Coluna: x
Quantidade de outliers: 32

Coluna: y
Quantidade de outliers: 29

Coluna: z
Quantidade de outliers: 49

Resumo estatístico do DataFrame após remoção dos outliers:
              carat  cut  clarity         depth         table         price  \
count  53891.000000  0.0      0.0  53891.000000  53891.000000  53891.000000   
mean       0.796432  NaN      NaN     61.748270     57.456752   3925.794734   
std        0.470301  NaN      NaN      1.430841      2.233780   3980.728263   
min        0.200000  NaN      NaN     43.000000     43.000000    326.000000   
25%        0.400000  NaN      NaN     61.000000     56.000000    949.000000   
50%        0.700000  NaN      NaN     61.800000     57.000000   2400.000000   
75%        1.040000  NaN      NaN     62.500000     59.000000   5316.000000   
max        3.050000 

In [ ]:
## 2. Tratamento de dados faltantes

In [11]:
# Verificando dados faltantes
print(f"Verificando dados faltante")
print(df.isnull().sum())

# Remover colunas com muitos dados faltantes
df = df.dropna(axis=1, thresh=int(0.9*len(df)))  # Remover colunas com mais de 10% de valores faltantes

# Imputar valores faltantes nas colunas numéricas com a mediana
df['carat'] = df['carat'].fillna(df['carat'].median())
df['depth'] = df['depth'].fillna(df['depth'].median())
df['table'] = df['table'].fillna(df['table'].median())
df['price'] = df['price'].fillna(df['price'].median())
df['x'] = df['x'].fillna(df['x'].median())
df['y'] = df['y'].fillna(df['y'].median())
df['z'] = df['z'].fillna(df['z'].median())

# Verificando novamente os dados faltantes
print(f"Verificando novamente os dados faltantes após o tratamento")
print(df.isnull().sum())

Verificando dados faltante
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64
Verificando novamente os dados faltantes após o tratamento
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64


In [ ]:
## 3. Codificação de variáveis

In [15]:
# Função para codificação de variáveis qualitativas
def encode_qualitative_variables(df):
    # Codificação de variáveis ordinais
    ordinal_mapping = {
        'cut': ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],  # ordem de qualidade
        'clarity': ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']  # ordem de clareza
    }

    for col, order in ordinal_mapping.items():
        df[col] = pd.Categorical(df[col], categories=order, ordered=True).codes

    # Codificação de variáveis nominais com One-Hot Encoding
    df = pd.get_dummies(df, columns=['color'], drop_first=True)

    return df

# Aplicar a função de codificação no DataFrame
df_encoded = encode_qualitative_variables(df)

# Verificando as primeiras linhas do DataFrame codificado
print(df_encoded.head())

   carat  cut  clarity  depth  table  price     x     y     z  color_E  \
0   0.23   -1       -1   61.5   55.0    326  3.95  3.98  2.43        1   
1   0.21   -1       -1   59.8   61.0    326  3.89  3.84  2.31        1   
2   0.23   -1       -1   56.9   65.0    327  4.05  4.07  2.31        1   
3   0.29   -1       -1   62.4   58.0    334  4.20  4.23  2.63        0   
4   0.31   -1       -1   63.3   58.0    335  4.34  4.35  2.75        0   

   color_F  color_G  color_H  color_I  color_J  
0        0        0        0        0        0  
1        0        0        0        0        0  
2        0        0        0        0        0  
3        0        0        0        1        0  
4        0        0        0        0        1  
